In [55]:
import torch

import pandas as pd
import numpy as np
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests 
import itertools
import spotlight
import tagme
import inflect
p = inflect.engine()
import re

from nltk.stem.porter import *
stemmer = PorterStemmer()

tagme.GCUBE_TOKEN = ""

In [149]:
with open('templates.json') as f:
    templates = json.load(f)

with open('data/lc-quad/test/id.txt') as f:
    ids = f.read().strip().split('\n')

# Prediction k-templates
saved_model = torch.load('checkpoints/Corrected spelling mistakes,epoch=47,test_acc=0.8246887966804979.pt')
trainer = saved_model['trainer']
test_dataset = torch.load('data/lc-quad/pth/lc_quad_test.pth')

def preprocess_relations(file, prop=False):
    relations = {}
    with open(file) as f:
        content = f.readlines()
        for line in content:
            split_line = line.split()

            key = ' '.join(split_line[2:])[1:-3].lower()
            key = ' '.join([stemmer.stem(word) for word in key.split()])
            
            if key not in relations:
                relations[key] = []
            
            uri = split_line[0].replace('<', '').replace('>', '')
            
            if prop is True:                
                uri_property = uri.replace('/ontology/', '/property/')
                relations[key].extend([uri, uri_property])
            else:
                relations[key].append(uri)                
    return relations    

properties = preprocess_relations('dbpedia_3Eng_property.ttl', True)
classes = preprocess_relations('dbpedia_3Eng_class.ttl')

In [10]:
questions = []
y_true = []
y_pred = []
output_vocab = trainer.vocabs['output']

for index in range(len(test_dataset)):
    torch.no_grad()    
    _, toks_sent, _, _, _ = test_dataset[index]
    tree, emb, target = trainer.get_data(test_dataset[index], test_dataset.num_classes)
    output = trainer.model.forward(tree, emb, training=False)
    _, pred = torch.topk(output[0].squeeze(0), 2)
    
    pred = pred.numpy()
    target = target.numpy()
    
    pred_0 = output_vocab.idxToLabel[pred[0]]
    pred_1 = output_vocab.idxToLabel[pred[1]]
    target = output_vocab.idxToLabel[target[0]]    
    
    questions.append(' '.join([trainer.vocabs['toks'].idxToLabel[int(index)] for index in toks_sent]))
    y_true.append(target)
    y_pred.append([pred_0, pred_1])


In [11]:
# Sorts based on descending order of values
def sort_dict_by_values(dictionary):
    keys = []
    for key, value in sorted(dictionary.items(), key=lambda item: (item[1], item[0]), reverse=True):
        keys.append(key)
    return keys

def get_earl_entities(query):
    THRESHOLD = 0.0001
    MAX_RESOURCES = 3
    MAX_PREDICATES = 5
    MAX_CLASSES = 3
    
    response = requests.post('http://sda.tech/earl/api/processQuery', 
                             json={"nlquery": query, "pagerankflag": False})
    
    json_response = json.loads(response.text)
    r_dict = {}
    c_dict = {}
    p_dict = {}
    
    reranked_lists = json_response['rerankedlists']
    for key in reranked_lists.keys():
        for result in reranked_lists[key]:
            if result[0] < THRESHOLD:
                continue                
            
            if result[1].startswith('http://dbpedia.org/resource/'):
                r_dict[result[1]] = result[0]
            elif result[1].startswith('http://dbpedia.org/'): 
                # Add to Classes
                if result[1].split('/')[-1][0].isupper(): 
                    c_dict[result[1]] = result[0]
                # Add to Predicates
                else:
                    p_dict[result[1]] = result[0]
        
    return {
        'r': sort_dict_by_values(r_dict)[:MAX_RESOURCES], 
        'p': sort_dict_by_values(p_dict)[:MAX_PREDICATES], 
        'c': sort_dict_by_values(c_dict)[:MAX_CLASSES]
    }

In [12]:
def get_tag_me_entities(query):
    MAX_ENTITIES = 5
    results = []
    response = requests.get("https://tagme.d4science.org/tagme/tag?lang=en&gcube-token={}&text={}"
                           .format('1b4eb12e-d434-4b30-8c7f-91b3395b96e8-843339462', query))
    
    annotations = {}
    for annotation in json.loads(response.text)['annotations']:        
        annotations['http://dbpedia.org/resource/' + annotation['title'].replace(' ', '_')] = annotation['rho']
    return sort_dict_by_values(annotations)[:MAX_ENTITIES]

In [13]:
def get_nliwod_entities(query, hashmap, include_properties = False):
#     ignore_list = ['name', 'list']
    ignore_list = []
    entities = []
    singular_query = [stemmer.stem(word) if p.singular_noun(word) == False else stemmer.stem(p.singular_noun(word)) for word in query.lower().split(' ')]
    
    for key in hashmap.keys():
        if key in ' '.join(singular_query) and len(key) > 2 and key not in ignore_list:
            entities += hashmap[key]
    return list(set(entities))

In [14]:
def get_entities(query):    
    return {
        'r': get_tag_me_entities(query),
        'p': get_nliwod_entities(query, properties, True),
        'c': get_nliwod_entities(query, classes)
    }

In [180]:
def get_spotlight_entities(query):
    entities = []
    try:
        annotations = spotlight.annotate('http://model.dbpedia-spotlight.org/en/annotate', query, confidence=0.4)    
        for annotation in annotations:
            entities.append(annotation['URI'])
    except:
        pass
    return entities

In [70]:
def make_sparql_query(query, return_var):
    sparql = SPARQLWrapper("http://akswnc9.aksw.uni-leipzig.de/dbpedia/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    print('\n\n', query, '\n\n')
    
    results = sparql.query().convert()
    
    if return_var == 'boolean':
        if 'boolean' in results:            
            return results['boolean']
        else:
            return None
    
    output = []
    for result in results["results"]["bindings"]:
        if return_var in result:
            output.append(result[return_var]['value'])
    return output


def get_rdfs_label(prop):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""
    SELECT ?label WHERE { <""" + prop + """> rdfs:label ?label . FILTER(lang(?label) = 'en') }
    """)
    sparql.setReturnFormat(JSON)    
    results = sparql.query().convert()
    
    for result in results["results"]["bindings"]:
        return result['label']['value']
    return None

In [16]:
def get_answer(question, template_id, strategy=None):
    output = []
    sparql_query = ''
    
    if strategy == 'earl':
        entities = get_earl_entities(question)
    else:
        entities = get_entities(question)
    
    template = templates[str(template_id)]
    slots = {}
    for slot in template['slots']:
        slots[slot] = entities[slot[0]]        
    
    # This means something probably went wrong and no predicates or resources were detected for the query
    if len(slots['p']) == 0 or len(slots['r']) == 0:
        return output, sparql_query
    
    ranges = []
    slot_keys = list(slots.keys())
    slot_len = len(slot_keys)
    for slot in slot_keys:
        ranges.append(range(len(slots[slot]))) 
        
    for i in itertools.product(*ranges):
        if slot_keys[0] == 'p' and slot_keys[1] == 'p2' and slots['p'][i[0]] == slots['p2'][i[1]]:
            continue
        if slot_keys[-2] == 'r' and slot_keys[-1] == 'r2' and slots['r'][i[-2]] == slots['r2'][i[-1]]:
            continue
            
        sparql_query = template['sparql']                
        for index in range(slot_len):
            sparql_query = sparql_query.replace('<' + slot_keys[index] + '>', '<' + slots[slot_keys[index]][i[index]] + '>')

        output = make_sparql_query(sparql_query, template['return'])        
        if template['return'] == 'boolean':
            if output == None:
                output = []
            else:
                output = [output]
        elif len(output) > 0:
            break
    
    return output, sparql_query

In [ ]:
start = 0
for index in range(start, len(questions)):
    question = questions[index]
    template = y_true[index]
    
    print(question, template)
    print('\n' * 2)
    output, sparql_query = get_answer(question, template)
    
    print(output, sparql_query)
    print('\n' * 4)

In [17]:
df = pd.read_csv('data/lc-quad/dataset.csv')
def get_entities_from_answer(id):
    string = df[df['_id'] == int(id)]['sparql_query'].tolist()[0]    
    matches = re.findall('<[^>]*>', string)
    matches = [uri.replace('<', '').replace('>', '') for uri in matches if uri != '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>']
    
    results = {'r': set(), 'p': set(), 'c': set()}    
    for match in matches:
        if match.startswith('http://dbpedia.org/resource/'):
            results['r'].add(match)
        elif match.split('/')[-1][0].isupper():
            results['c'].add(match)
        else:
            results['p'].add(match)
    
    return results

In [182]:
start = 0
exists = 0
count = 1
results = []
res_df = pd.read_csv('slot_results.csv')

for index in range(start, len(questions)):
    df_row = res_df.loc[index,:]

    if df_row['Correct'] == True:
        continue
        
    if df_row['C_R'] != df_row['N_R'] and df_row['C_P'] == df_row['N_P']:
        question = questions[index]
        template = y_true[index]

        print('\n' * 2)    
        print(index, question, template)

        entities_list = get_entities_from_answer(ids[index])
        detected_entities = get_earl_entities(question)
        detected_entities['r'] = get_tag_me_entities(question) + get_spotlight_entities(question)

        for e_index in range(len(detected_entities['p'])): # Adding property namespace to Earl predicates
            entity = detected_entities['p'][e_index]
            detected_entities['p'].append(entity.replace('/ontology/', '/property/'))

        detected_entities['p'] += get_nliwod_entities(question, properties)

        for e_index in range(len(detected_entities['p'])): # Adding plural properties to all detected predicates
            entity = detected_entities['p'][e_index]
            if entity.startswith('http://dbpedia.org/property/'):
                pred = entity.split('/')[-1].split('_')
                if len(pred) == 1:
                    detected_entities['p'].append('http://dbpedia.org/property/' + p.plural_noun(pred[0]))

        detected_entities['r'] = set(detected_entities['r'])
        detected_entities['p'] = set(detected_entities['p'])

        unique_p = set()
        for entity in detected_entities['p']:
            e = p.singular_noun(entity.split('/')[-1])
            if e is not False:
                unique_p.add(e)

        for entity in unique_p:
            if entity == entity.lower():
                detected_entities['c'].append('http://dbpedia.org/ontology/' + entity[0].upper() + entity[1:])        
        detected_entities['c'] = set(detected_entities['c'])

        row = [len(entities_list['r']), len(entities_list['p']), len(entities_list['c']),
               len(detected_entities['r']), len(unique_p), len(detected_entities['c']),

               len(entities_list['r'].intersection(detected_entities['r'])),
               len(entities_list['p'].intersection(detected_entities['p'])),
               len(entities_list['c'].intersection(detected_entities['c']))
              ]
        row.append(row[0] == row[-3] and row[1] == row[-2] and row[2] == row[-1]) # Check if number and correct are the same    
        res_df.loc[index,:] = row

        print('\n', entities_list['r'] - detected_entities['r'], '\n', entities_list['p'] - detected_entities['p'])    
        print(index, row)            




6 which operator of ganz uv is also the nationality of the gza horvth ? 16

 {'http://dbpedia.org/resource/Géza_Horváth'} 
 set()
6 [2, 2, 0, 4, 8, 8, 1, 2, 0, False]



13 who is the meaning of the name isa which was also one of the subject of play the prodigal son ? 16

 {'http://dbpedia.org/resource/The_Prodigal_Son_(play)', 'http://dbpedia.org/resource/Isa_(name)'} 
 set()
13 [2, 2, 0, 6, 12, 10, 0, 2, 0, False]



14 how many schools have a bison as their mascot ? 101

 {'http://dbpedia.org/resource/Bison'} 
 set()
14 [1, 1, 1, 3, 6, 8, 0, 1, 1, False]



15 which currency of republic of montenegro is also the currency of the french southern and antarctic lands ? 16

 {'http://dbpedia.org/resource/Republic_of_Montenegro_(1992–2006)'} 
 set()
15 [2, 2, 0, 4, 7, 7, 1, 2, 0, False]



19 which company ownes stock in dagenham turbines and ford motor credit ? 15

 {'http://dbpedia.org/resource/Dagenham_wind_turbines'} 
 set()
19 [2, 1, 0, 7, 13, 14, 1, 1, 0, False]



22 is george l


 {'http://dbpedia.org/resource/Mark_Stevens_(film_editor)'} 
 set()
323 [1, 1, 1, 6, 8, 11, 0, 1, 1, False]



328 list the institute of robert hall ? 2

 {'http://dbpedia.org/resource/Robert_Hall_(economist)'} 
 set()
328 [1, 1, 0, 3, 6, 6, 0, 1, 0, False]



334 what are breann mcgregor and anika knudsen , both known for ? 15

 {'http://dbpedia.org/resource/Anika_Knudsen'} 
 set()
334 [2, 1, 0, 3, 10, 8, 1, 1, 0, False]



344 name the fields of paul demiville ? 2

 {'http://dbpedia.org/resource/Paul_Demiéville'} 
 set()
344 [1, 1, 0, 3, 6, 9, 0, 1, 0, False]



347 where are the movies directed by faisal aman khan recorded ? 5

 {'http://dbpedia.org/resource/Faisal_Aman_Khan'} 
 set()
347 [1, 2, 0, 7, 10, 8, 0, 2, 0, False]



353 who developed games based on the cars series ? 5

 {'http://dbpedia.org/resource/Cars_(film)'} 
 set()
353 [1, 2, 1, 3, 14, 14, 0, 2, 0, False]



358 is angola avante the anthem of angola ? 151

 set() 
 set()
358 [2, 1, 0, 3, 5, 8, 2, 1, 0, True]



360


 {'http://dbpedia.org/resource/Masahiko_Minami'} 
 set()
684 [1, 2, 1, 5, 13, 15, 0, 2, 1, False]



686 which people are known for appearing on the television show dragon 's den ? 1

 {"http://dbpedia.org/resource/Dragons'_Den_(UK_TV_series)"} 
 set()
686 [1, 1, 1, 5, 12, 12, 0, 1, 1, False]



692 who appointed joseph clay and william cushing to their office ? 15

 {'http://dbpedia.org/resource/Joseph_Clay_(Georgia)'} 
 set()
692 [2, 1, 0, 4, 7, 10, 1, 1, 0, False]



723 does chicago own the washington station on the cta blue line ? 151

 {'http://dbpedia.org/resource/Washington_station_(CTA_Blue_Line)'} 
 set()
723 [2, 1, 0, 5, 13, 12, 1, 1, 0, False]



742 which leader of united states house representatives election was married to dolley madison ? 16

 {'http://dbpedia.org/resource/United_States_House_of_Representatives_elections,_1788_and_1789'} 
 set()
742 [2, 2, 0, 5, 16, 17, 1, 2, 0, False]



781 which awards have been given to scientists that graduated from nku athens ? 5


In [197]:

416 does the merlin express have a hub at the cibano international airport ? 151  {'http://dbpedia.org/resource/Cibao_International_Airport'} 

444 who is the writer of neverwhere ( radio play ) ? 2  {'http://dbpedia.org/resource/Neverwhere_(radio_play)'} 

474 which partner of the tim mathieson is also the primeminister of the mike kelly ( australian politician ) ? 16  {'http://dbpedia.org/resource/Mike_Kelly_(Australian_politician)'} 

577 what show has theme music composer as ron grainer and at the same time related to class ( 2016 tv series ) ? 8  {'http://dbpedia.org/resource/Class_(2016_TV_series)'} 


587 what are shows whose theme music composer 's home town is new york ? 6  {'http://dbpedia.org/resource/New_York'} 
    
636 is latin the official language of sovereign military order of malta ? 151  {'http://dbpedia.org/resource/Latin'} 

789 name the champion of national league ? 2  {'http://dbpedia.org/resource/National_League'} 

874 what is the nickname of the club managed by piotr baron ? 5  {'http://dbpedia.org/resource/Piotr_Baron'} 






503 who is the operator of rainbow warrior ( 1955 ) ? 2  {'http://dbpedia.org/resource/Rainbow_Warrior_(1955)'} 



577 what show has theme music composer as ron grainer and at the same time related to class ( 2016 tv series ) ? 8




455 is esther alder the mayor of geneva ? 151  {'http://dbpedia.org/resource/Geneva'} 


SyntaxError: invalid syntax (<ipython-input-197-2379a1420aed>, line 2)

In [216]:
def get_spotlight_entities(query):
    entities = []
    try:
        annotations = spotlight.annotate('http://model.dbpedia-spotlight.org/en/annotate', query, confidence=0.0)    
        for annotation in annotations:
            entities.append(annotation['URI'])
    except:
        pass
    return entities

get_spotlight_entities("Does the merlin express have a hub at the cibano international airport?")

['http://dbpedia.org/resource/John_Doe',
 'http://dbpedia.org/resource/Merlin_Express',
 'http://dbpedia.org/resource/Airline_hub',
 'http://dbpedia.org/resource/International_airport']

In [94]:
# res_df = pd.DataFrame(results, columns=['N_R', 'N_P', 'N_C', 'D_R', 'D_P', 'D_C', 'C_R', 'C_P', 'C_C', 'Correct'])
# for i in range(len(res_df)):
#     row = res_df.loc[i,:]
#     res_df.loc[i, 'Correct'] = row['N_R'] == row['C_R'] and row['N_P'] == row['C_P'] and row['N_C'] == row['C_C']

# res_df.to_csv('slot_results.csv', index=False)

In [50]:
targets = res_df[res_df['N_P'] - res_df['C_P'] == 1].index.tolist()

In [72]:
# with open('props.csv') as f:
#     props = f.read().split('\n')
# props = props[1:]

# string = ''
# for prop in props:
#     label = get_rdfs_label(prop)
#     if label is not None:
#         print(prop, label)
#         string += '<' + prop + '>' + '<http://www.w3.org/2000/01/rdf-schema#label> "' + label + '" .\n'
        
# with open('props.txt', 'w') as f:
#     f.write(string)

http://dbpedia.org/property/abbreviation abbreviation
http://dbpedia.org/property/absMagnitude abs magnitude
http://dbpedia.org/property/academicAdvisors academic advisors
http://dbpedia.org/property/access access
http://dbpedia.org/property/acreage acreage
http://dbpedia.org/property/act act
http://dbpedia.org/property/ada ADA
http://dbpedia.org/property/added added
http://dbpedia.org/property/address address
http://dbpedia.org/property/addresses addresses
http://dbpedia.org/property/adjacentcomuni adjacentcomuni
http://dbpedia.org/property/administrator administrator
http://dbpedia.org/property/aEngineer a engineer
http://dbpedia.org/property/affiliates affiliates
http://dbpedia.org/property/affiliation affiliation
http://dbpedia.org/property/affiliations affiliations
http://dbpedia.org/property/age age
http://dbpedia.org/property/agency agency
http://dbpedia.org/property/agencyName agency name
http://dbpedia.org/property/agencyType agency type
http://dbpedia.org/property/ageRange ag

http://dbpedia.org/property/beds Beds
http://dbpedia.org/property/beganOperation began operation
http://dbpedia.org/property/begin begin
http://dbpedia.org/property/belfastDistance belfast distance
http://dbpedia.org/property/belfastDistanceKm belfast distance km
http://dbpedia.org/property/belfastDistanceMi belfast distance mi
http://dbpedia.org/property/beltwayCity beltway city
http://dbpedia.org/property/bestFinish Best Finish
http://dbpedia.org/property/bezirk Bezirk
http://dbpedia.org/property/bicycle bicycle
http://dbpedia.org/property/billed billed
http://dbpedia.org/property/binomial binomial
http://dbpedia.org/property/binomialAuthority binomial authority
http://dbpedia.org/property/bioavailability bioavailability
http://dbpedia.org/property/biome biome
http://dbpedia.org/property/birth birth
http://dbpedia.org/property/birthdate birthdate
http://dbpedia.org/property/birthDate birth date
http://dbpedia.org/property/birthname birthname
http://dbpedia.org/property/birthName birt

http://dbpedia.org/property/cityType city type
http://dbpedia.org/property/class class
http://dbpedia.org/property/classes classes
http://dbpedia.org/property/classesOffered classes offered
http://dbpedia.org/property/classis classis
http://dbpedia.org/property/classrooms classrooms
http://dbpedia.org/property/close close
http://dbpedia.org/property/closed closed
http://dbpedia.org/property/closedDate closed date
http://dbpedia.org/property/closing closing
http://dbpedia.org/property/closingDate closing date
http://dbpedia.org/property/clposition clposition
http://dbpedia.org/property/club club
http://dbpedia.org/property/clubname clubname
http://dbpedia.org/property/clubnumber clubnumber
http://dbpedia.org/property/clubs clubs
http://dbpedia.org/property/clyears clyears
http://dbpedia.org/property/coach coach
http://dbpedia.org/property/coachingRecords coaching records
http://dbpedia.org/property/coachTeams coach teams
http://dbpedia.org/property/coachyear1end coachyear1end
http://dbp

http://dbpedia.org/property/denomination denomination
http://dbpedia.org/property/density density
http://dbpedia.org/property/densityKm Density km
http://dbpedia.org/property/densitySqMi density sq mi
http://dbpedia.org/property/department department
http://dbpedia.org/property/departments departments
http://dbpedia.org/property/deployed deployed
http://dbpedia.org/property/depth depth
http://dbpedia.org/property/depthM depth m
http://dbpedia.org/property/deputy deputy
http://dbpedia.org/property/deputyHead deputy head
http://dbpedia.org/property/derivatives derivatives
http://dbpedia.org/property/description Description
http://dbpedia.org/property/design design
http://dbpedia.org/property/designatedOther1Name designated other1 name
http://dbpedia.org/property/designatedOther2Name designated other2 name
http://dbpedia.org/property/designer designer
http://dbpedia.org/property/destinations destinations
http://dbpedia.org/property/destructionDate destruction date
http://dbpedia.org/prope

http://dbpedia.org/property/fail fail
http://dbpedia.org/property/failed failed
http://dbpedia.org/property/fam fam
http://dbpedia.org/property/fame fame
http://dbpedia.org/property/familia familia
http://dbpedia.org/property/family family
http://dbpedia.org/property/fansgroup fansgroup
http://dbpedia.org/property/fareZone fare zone
http://dbpedia.org/property/fastCountry Fast Country
http://dbpedia.org/property/fastDriver Fast Driver
http://dbpedia.org/property/fastestLaps Fastest laps
http://dbpedia.org/property/fastTeam Fast Team
http://dbpedia.org/property/fat fat
http://dbpedia.org/property/fate fate
http://dbpedia.org/property/father father
http://dbpedia.org/property/feastDay feast day
http://dbpedia.org/property/fees fees
http://dbpedia.org/property/field field
http://dbpedia.org/property/fields fields
http://dbpedia.org/property/filename filename
http://dbpedia.org/property/films films
http://dbpedia.org/property/finalChampion final champion
http://dbpedia.org/property/finalda

...

http://dbpedia.org/property/party party
http://dbpedia.org/property/partyElection party election
http://dbpedia.org/property/parvordo parvordo
http://dbpedia.org/property/passengers passengers
http://dbpedia.org/property/passPercent pass percent
http://dbpedia.org/property/passSystem pass system
http://dbpedia.org/property/passYear pass year
http://dbpedia.org/property/pastmembers pastmembers
http://dbpedia.org/property/pastMembers past members
http://dbpedia.org/property/pastor pastor
http://dbpedia.org/property/pastschools pastschools
http://dbpedia.org/property/pastteams pastteams
http://dbpedia.org/property/patron patron
http://dbpedia.org/property/pdb PDB
http://dbpedia.org/property/peerReviewed peer-reviewed
http://dbpedia.org/property/penname penname
http://dbpedia.org/property/percentWater percent water
http://dbpedia.org/property/periapsis periapsis
http://dbpedia.org/property/periastron periastron
http://dbpedia.org/property/perihelion perihelion
http://dbpedia.org/property/p

http://dbpedia.org/property/published published
http://dbpedia.org/property/publisher publisher
http://dbpedia.org/property/publisherEn publisher en
http://dbpedia.org/property/pupils pupils
http://dbpedia.org/property/purpose purpose
http://dbpedia.org/property/r R
http://dbpedia.org/property/r1LengthF r1-length-f
http://dbpedia.org/property/r1LengthM r1-length-m
http://dbpedia.org/property/r1Number r1-number
http://dbpedia.org/property/r1Surface r1-surface
http://dbpedia.org/property/r2LengthF r2-length-f
http://dbpedia.org/property/r2LengthM r2-length-m
http://dbpedia.org/property/r2Number r2-number
http://dbpedia.org/property/r2Surface r2-surface
http://dbpedia.org/property/r3LengthF r3-length-f
http://dbpedia.org/property/r3LengthM r3-length-m
http://dbpedia.org/property/r3Number r3-number
http://dbpedia.org/property/r3Surface r3-surface
http://dbpedia.org/property/r4LengthF r4-length-f
http://dbpedia.org/property/r4LengthM r4-length-m
http://dbpedia.org/property/r4Number r4-numbe

http://dbpedia.org/property/shipChristened Ship christened
http://dbpedia.org/property/shipClass Ship class
http://dbpedia.org/property/shipCommissioned Ship commissioned
http://dbpedia.org/property/shipCompleted Ship completed
http://dbpedia.org/property/shipCountry Ship country
http://dbpedia.org/property/shipDecommissioned Ship decommissioned
http://dbpedia.org/property/shipDisplacement Ship displacement
http://dbpedia.org/property/shipDraft Ship draft
http://dbpedia.org/property/shipFate Ship fate
http://dbpedia.org/property/shipHeight Ship height
http://dbpedia.org/property/shipHomeport Ship homeport
http://dbpedia.org/property/shipInService Ship in service
http://dbpedia.org/property/shipLaidDown Ship laid down
http://dbpedia.org/property/shipLaunched Ship launched
http://dbpedia.org/property/shipLength Ship length
http://dbpedia.org/property/shipMaidenVoyage Ship maiden voyage
http://dbpedia.org/property/shipMotto Ship motto
http://dbpedia.org/property/shipName Ship name
http://

http://dbpedia.org/property/subRegions sub regions
http://dbpedia.org/property/subregnum subregnum
http://dbpedia.org/property/subsid subsid
http://dbpedia.org/property/subsidiaries subsidiaries
http://dbpedia.org/property/subspecies subspecies
http://dbpedia.org/property/subtitle subtitle
http://dbpedia.org/property/subtribus subtribus
http://dbpedia.org/property/succeeded succeeded
http://dbpedia.org/property/succeeding succeeding
http://dbpedia.org/property/success success
http://dbpedia.org/property/successor successor
http://dbpedia.org/property/sudoc SUDOC
http://dbpedia.org/property/summerappearances summerappearances
http://dbpedia.org/property/superclassis superclassis
http://dbpedia.org/property/superdivisio superdivisio
http://dbpedia.org/property/superdomain superdomain
http://dbpedia.org/property/superfamilia superfamilia
http://dbpedia.org/property/superintendent superintendent
http://dbpedia.org/property/superordo superordo
http://dbpedia.org/property/superphylum superph

http://dbpedia.org/property/verbandsgemeinde Verbandsgemeinde
http://dbpedia.org/property/verwaltungsgemeinschaft Verwaltungsgemeinschaft
http://dbpedia.org/property/verwaltungsverband Verwaltungsverband
http://dbpedia.org/property/viaf viaf
http://dbpedia.org/property/viceChancellor Vice-Chancellor
http://dbpedia.org/property/vicepresident vicepresident
http://dbpedia.org/property/vicePresident Vice President
http://dbpedia.org/property/viceprimeminister viceprimeminister
http://dbpedia.org/property/viceprincipal viceprincipal
http://dbpedia.org/property/viceprincipalLabel viceprincipal label
http://dbpedia.org/property/vineyards vineyards
http://dbpedia.org/property/virtual virtual
http://dbpedia.org/property/visitation visitation
http://dbpedia.org/property/visitationNum visitation num
http://dbpedia.org/property/visitationYear visitation year
http://dbpedia.org/property/visitors visitors
http://dbpedia.org/property/voice voice
http://dbpedia.org/property/voicedBy voiced by
http://d

In [183]:
res_df['Correct'].value_counts()

False    526
True     438
Name: Correct, dtype: int64

In [155]:
res_df.to_csv('slot_results.csv', index=False)

In [184]:
correct = res_df['Correct']
accuracy = 0
top_2_accuracy = 0

for index in range(len(questions)):
    if correct[index] == True:
        if y_true[index] == y_pred[index][0]:
            accuracy += 1
            top_2_accuracy += 1
        elif y_true[index] == y_pred[index][1]:
            top_2_accuracy += 1
accuracy / len(correct), top_2_accuracy / len(correct), correct.sum() / len(correct)

(0.3879668049792531, 0.4263485477178423, 0.45435684647302904)